In [ ]:
# Framework Validation 25.5.2020 - Elishai
from neuron import h
from SimParameters import SimulationParameters
from Simulation import Simulation


refiling_rate = 0.06336561569006663 
release_probability = 0.06336561569006663
offset_dyn = 0.01029862447592878
scale_factor = 0.33364277766404854
offset = 0.5250708508639323
transition_placement = 105
syn_weight = 2.7849705608729593e-06
light_dt = 0.2

############################## EXPANDING ######################
simulation_parameters = SimulationParameters("simulation_parameters/validation_network_exp.xml",
                                              scale_factor, offset, transition_placement, refiling_rate, 
                                              release_probability, offset_dyn, syn_weight,light_dt=light_dt,
                                              note='framework validation network') 

exp_simulation = Simulation(simulation_parameters)

exp_simulation.simulate()
exp_results = exp_simulation.trace_voltages_at_soma()
exp_simulation.visualize_results()
exp_simulation.simulation['pc'].gid_clear()

del simulation_parameters
del exp_simulation
h("forall {delete_section()}")

############################## COLLAPSING ######################

simulation_parameters = SimulationParameters("simulation_parameters/validation_network_col.xml",
                                              scale_factor, offset, transition_placement, refiling_rate, 
                                              release_probability, offset_dyn, syn_weight,light_dt=light_dt,
                                              note='framework validation network') 

col_simulation = Simulation(simulation_parameters)
col_simulation.simulate()
col_results = col_simulation.trace_voltages_at_soma()
col_simulation.visualize_results()

No handles with labels found to put in legend.
/Users/elishai/Dropbox/1AFiles/Updated VSME/VSME_GA/Functions.py:1101: UserWarning: The following kwargs were not used by contour: 'interpolation'
  cf = ax.contourf(X, Y, a, cmap = 'gist_gray', interpolation='nearest')


In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(8,5))
for i in range(1,14):
    plt.plot(exp_results[0]['0'][i]['t'], exp_results[0]['0'][i]['v'], ls='-', c='r', label='Exp')
    plt.plot(col_results[0]['0'][i]['t'], col_results[0]['0'][i]['v'], ls='-', c='b', label='Col')
    plt.legend()
    plt.title(i)
    # plt.savefig('Res_cell_{}.jpg'.format(i), dpi=350, format='jpg')
    plt.show()

In [ ]:
exp_results[0]['0'].keys()


In [ ]:
xs = []
ys = []
sps = []


for cell in col_simulation.synapse_dict['light']['0']:
    for i in range(len(col_simulation.synapse_dict['light']['0'][cell]['x'])):
        x = col_simulation.synapse_dict['light']['0'][cell]['x'][i]
        y = col_simulation.synapse_dict['light']['0'][cell]['y'][i]
        syn = col_simulation.synapse_dict['light']['0'][cell]['BC_syn'][i]
        spike_times = col_simulation.simulation['spikes']['0'][cell][i]
        for sp_time in spike_times:
            xs.append(x)
            ys.append(y)
            sps.append(sp_time)

import seaborn as sns
import pandas as pd
from matplotlib import animation, rc
from IPython.display import HTML
from neuron import h
import matplotlib as mpl
mpl.rcParams['animation.embed_limit']= 100
%pylab
%matplotlib inline
df = pd.DataFrame({'x':xs,'y':ys,'sps':sps})
df.index = df.sps
df = df.sort_index()

fig = figure(figsize=(9,9))
ax1 = subplot(1,1,1)
color = sns.color_palette("Paired",14)
for cell in col_simulation.simulation['cells']['0']['instances']:
    for sec in col_simulation.simulation['cells']['0']['instances'][cell]['section_dict']['section_list']:
        xs2 =[]
        ys2 =[]
        for i in range(int(sec.n3d())):
            xs2.append(sec.x3d(i))
            ys2.append(sec.y3d(i))
        plot(xs2, ys2,color=color[cell-1])
    plt.axis('equal')
#xlim(min(df.x),max(df.x))
#ylim(min(df.y),max(df.y))
# ax1.set_axis_off()

old_sps =scatter([0],[0],marker='o')
def show_spikes(dt):
    global time, old_sps
    title(f't = {time} ms')
    sps = df.loc[time:time+dt]
    old_sps.remove()
    scs = scatter(sps.x,sps.y,marker='o', color='k')
    old_sps = scs
    time +=dt
    return []

time = 0
anim = animation.FuncAnimation(fig, show_spikes, frames=[1] + [4]*200)

Writer = animation.writers['ffmpeg']
writer = Writer(fps=15, metadata=dict(artist='Me'), bitrate=1800)
anim.save('im.mp4', writer=writer)
HTML(anim.to_jshtml())

In [ ]:

sns.palplot(
